# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Loading the dotenv library
%load_ext dotenv

# Loading the env variables
%dotenv ../src/.env

In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [13]:
import os
from glob import glob

# loading PRICE_DATA environment variable
PRICE_DATA = os.getenv("PRICE_DATA")

# using glob to find path to all parquet files
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))

In [14]:
parquet_files

['../data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2002.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2003.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2004.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2005.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2006.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2007.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2008.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2009.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2010.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2011.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2012.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2013.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2014.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2015.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2016.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2017.parquet\\part.0.parquet',
 '../data/prices\\A\\A_2018.

In [15]:
# Reads Parquets into Dask Datagram but does not execute
dd_px = dd.read_parquet(parquet_files)

In [16]:
# Executes the read and produces Dask Dataframe
dd_px.compute()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.683865,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.346905,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.844181,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.403217,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.436810,2819626,Health Care,Life Sciences Tools & Services,2000
...,...,...,...,...,...,...,...,...,...,...
ZION,2024-02-26,38.610001,39.439999,38.040001,38.419998,38.419998,2030500,Financials,Regional Banks,2024
ZION,2024-02-27,38.860001,39.240002,38.340000,38.680000,38.680000,1786000,Financials,Regional Banks,2024
ZION,2024-02-28,38.290001,38.689999,38.009998,38.080002,38.080002,1487000,Financials,Regional Banks,2024


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [17]:
# Write your code below.
import numpy as np

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Adj_close_lag_1 = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_close_lag_1'] - 1
).assign(
    hi_lo_range = lambda x: (x['High'] - x['Low'])
))

C:\Users\bash\AppData\Local\Temp\ipykernel_29688\1538281725.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [18]:
# Converting a Dask dataframe to Pandas dataframe
pd_feat = dd_feat.compute()

c:\Users\bash\Documents\GitHub\production\env\Lib\site-packages\pandas\core\frame.py:706: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
c:\Users\bash\Documents\GitHub\production\env\Lib\site-packages\pandas\core\frame.py:706: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
c:\Users\bash\Documents\GitHub\production\env\Lib\site-packages\pandas\core\frame.py:706: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
c:\Users\bash\Documents\GitHub\production\env\Lib\site-packages\pandas\core\frame.py:706: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(
c:\Users\bash\Documents\GitHub\production\env\Li

In [19]:
type(pd_feat)

pandas.core.frame.DataFrame

In [20]:
# import pandas as pd
pd_feat = pd_feat.assign(rolling_mean = pd_feat['returns'].rolling(10).mean())

# pd_feat = pd_feat.assign(rolling_mean = lambda x: x['returns'].rolling(10).mean())

In [21]:
pd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Adj_close_lag_1,returns,hi_lo_range,rolling_mean
ticker,,,,,,,,,,,,,,
DFS,2010-01-04,14.870000,14.880000,14.500000,14.610000,11.240438,7552600,Financials,Consumer Finance,2010,NaN,NaN,0.380000,NaN
DFS,2010-01-05,14.800000,15.160000,14.650000,15.100000,11.617424,7766400,Financials,Consumer Finance,2010,11.240438,0.033538,0.510000,NaN
DFS,2010-01-06,14.850000,15.040000,14.850000,15.000000,11.540486,5550200,Financials,Consumer Finance,2010,11.617424,-0.006623,0.190000,NaN
DFS,2010-01-07,14.950000,15.180000,14.850000,15.090000,11.609728,6492500,Financials,Consumer Finance,2010,11.540486,0.006000,0.330000,NaN
DFS,2010-01-08,15.110000,15.150000,14.900000,15.030000,11.563570,4458300,Financials,Consumer Finance,2010,11.609728,-0.003976,0.250000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WRB,2020-12-24,44.099998,44.193333,43.446667,44.066666,41.095810,224250,Financials,Property & Casualty Insurance,2020,41.021214,0.001818,0.746666,0.002168
WRB,2020-12-28,44.253334,44.386665,43.919998,44.060001,41.089600,808950,Financials,Property & Casualty Insurance,2020,41.095810,-0.000151,0.466667,0.002230
WRB,2020-12-29,44.293331,44.326668,43.613335,43.820000,40.865776,597000,Financials,Property & Casualty Insurance,2020,41.089600,-0.005447,0.713333,0.002551


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Comments:
* Yes, it was necessary to covert from a Dask dataframe to Pandas dataframe. When attempting to calculate using Dask, it raises an Error stating "ValueError: Can only rolling dataframes with known divisions"
* Dask would be better for it's efficiency and speed.